<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/compoundoption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import QuantLib as ql
import unittest

def format_report_failure(greek_name, mother_option_type, daughter_option_type,
                          mother_strike, daughter_strike, spot_val, q_rate, r_rate,
                          today, mother_maturity_date, daughter_maturity_date,
                          volatility, expected, calculated, error, tolerance):
    return (
        f"\nFailure for {greek_name}:"
        f"\nMother option type:   {mother_option_type}"
        f"\nDaughter option type: {daughter_option_type}"
        f"\nspot value:           {spot_val}"
        f"\nstrike mother:        {mother_strike}"
        f"\nstrike daughter:      {daughter_strike}"
        f"\ndividend yield:       {q_rate:.4f}"
        f"\nrisk-free rate:       {r_rate:.4f}"
        f"\nreference date:       {ql.Date.ISOformat(today)}"
        f"\nmaturity mother:      {ql.Date.ISOformat(mother_maturity_date)}"
        f"\nmaturity daughter:    {ql.Date.ISOformat(daughter_maturity_date)}"
        f"\nvolatility:           {volatility:.4f}"
        f"\n  expected {greek_name}: {expected}"
        f"\ncalculated {greek_name}: {calculated}"
        f"\nerror:                {error}"
        f"\ntolerance:            {tolerance}"
    )

class CompoundOptionTests(unittest.TestCase):

    def setUp(self):
        """Set up common objects for the tests."""
        self.calendar = ql.TARGET()
        self.day_counter = ql.Actual360()

        # Using evaluation date from global settings, as TopLevelFixture would imply
        self.todays_date = ql.Settings.instance().evaluationDate

        self.spot_quote = ql.SimpleQuote(0.0)
        self.q_rate_quote = ql.SimpleQuote(0.0)
        self.r_rate_quote = ql.SimpleQuote(0.0)
        self.vol_quote = ql.SimpleQuote(0.0)

        self.spot_handle = ql.QuoteHandle(self.spot_quote)
        self.q_rate_handle = ql.QuoteHandle(self.q_rate_quote)
        self.r_rate_handle = ql.QuoteHandle(self.r_rate_quote)
        self.vol_handle = ql.QuoteHandle(self.vol_quote)

        self.r_ts_handle = ql.YieldTermStructureHandle(
            ql.FlatForward(0, ql.NullCalendar(), self.r_rate_handle, self.day_counter)
        )
        self.q_ts_handle = ql.YieldTermStructureHandle(
            ql.FlatForward(0, ql.NullCalendar(), self.q_rate_handle, self.day_counter)
        )
        self.vol_ts_handle = ql.BlackVolTermStructureHandle(
            ql.BlackConstantVol(self.todays_date, ql.NullCalendar(), self.vol_handle, self.day_counter)
        )

        self.process = ql.BlackScholesMertonProcess(
            self.spot_handle, self.q_ts_handle, self.r_ts_handle, self.vol_ts_handle
        )

        self.compound_engine = ql.AnalyticCompoundOptionEngine(self.process)
        self.european_engine = ql.AnalyticEuropeanEngine(self.process)

    def _update_market_data(self, s, q, r, v):
        self.spot_quote.setValue(s)
        self.q_rate_quote.setValue(q)
        self.r_rate_quote.setValue(r)
        self.vol_quote.setValue(v)

    def test_put_call_parity(self):
        """Testing compound-option put-call parity."""
        print("Testing compound-option put-call parity...")

        # Data from C++ test:
        # type Mother, typeDaughter, strike Mother, strike Daughter, spot, q, r, t Mother, t Daughter, vol
        parity_data = [
            { "sM": 50.0, "sD": 520.0, "s": 500.0, "q": 0.03, "r": 0.08, "tM": 0.25, "tD": 0.5, "v": 0.35},
            # C++ test has Call/Call with same params for parity, this row is redundant as mother type changes
            # but included for completeness if original C++ test used separate definition of `values`
            # The C++ test implies specific pairs for parity (CallOnX vs PutOnX)
            # The C++ test setup for parity actually tests:
            # CallOnDaughter(K_M) + K_M*exp(-r*T_M) - PutOnDaughter(K_M) - VanillaDaughter = 0
            # where K_M is mother strike, T_M is mother expiry. Daughter option is the same for all 3.

            # For the test, we'll iterate over parameters for the daughter option and mother strike/maturity
            # then construct CallOnDaughter and PutOnDaughter

            # Reinterpreting C++ test structure:
            # It iterates a list of daughter option params and mother option params (excluding mother type)
            # For each set, it constructs CallOnDaughter and PutOnDaughter for the parity check.

            # Parameters for the underlying (daughter) option and market conditions
            # Mother type is implicitly Call and Put for the parity relation.
            # strikeMother in C++ data is used for K_M in the parity.
            test_cases = [
                # DaughterType, strikeMother, strikeDaughter, spot, q, r, tMother, tDaughter, vol
                (ql.Option.Call, 50.0, 520.0, 500.0, 0.03, 0.08, 0.25, 0.5, 0.35),
                (ql.Option.Put,  50.0, 520.0, 500.0, 0.03, 0.08, 0.25, 0.5, 0.35), # Test with Put daughter
                (ql.Option.Call, 0.05, 1.14,  1.20,  0.0,  0.01, 0.5,  2.0, 0.11),
                (ql.Option.Put,  0.05, 1.14,  1.20,  0.0,  0.01, 0.5,  2.0, 0.11),
                (ql.Option.Call, 10.0, 122.0, 120.0, 0.06, 0.02, 0.1,  0.7, 0.22),
                (ql.Option.Put,  10.0, 122.0, 120.0, 0.06, 0.02, 0.1,  0.7, 0.22),
                (ql.Option.Call, 0.4,  8.2,   8.0,   0.05, 0.00, 2.0,  3.0, 0.08),
                (ql.Option.Put,  0.4,  8.2,   8.0,   0.05, 0.00, 2.0,  3.0, 0.08),
                (ql.Option.Call, 0.02, 1.6,   1.6,   0.013,0.022,0.45, 0.5, 0.17),
                (ql.Option.Put,  0.02, 1.6,   1.6,   0.013,0.022,0.45, 0.5, 0.17)
            ]
        ]

        for (daughter_type, strike_mother, strike_daughter, s, q, r, t_mother, t_daughter, v) in test_cases:
            self._update_market_data(s, q, r, v)

            payoff_mother_call = ql.PlainVanillaPayoff(ql.Option.Call, strike_mother)
            payoff_mother_put = ql.PlainVanillaPayoff(ql.Option.Put, strike_mother)
            payoff_daughter = ql.PlainVanillaPayoff(daughter_type, strike_daughter)

            # timeToDays(Time t) { return Natural(t*365.0); }
            maturity_mother_date = self.todays_date + int(t_mother * 365)
            maturity_daughter_date = self.todays_date + int(t_daughter * 365)

            exercise_mother = ql.EuropeanExercise(maturity_mother_date)
            exercise_daughter = ql.EuropeanExercise(maturity_daughter_date)

            compound_call_on_daughter = ql.CompoundOption(
                payoff_mother_call, exercise_mother, payoff_daughter, exercise_daughter
            )
            compound_put_on_daughter = ql.CompoundOption(
                payoff_mother_put, exercise_mother, payoff_daughter, exercise_daughter
            )
            vanilla_daughter_option = ql.VanillaOption(payoff_daughter, exercise_daughter)

            compound_call_on_daughter.setPricingEngine(self.compound_engine)
            compound_put_on_daughter.setPricingEngine(self.compound_engine)
            vanilla_daughter_option.setPricingEngine(self.european_engine)

            discount_factor = self.r_ts_handle.discount(maturity_mother_date)
            discounted_mother_strike = strike_mother * discount_factor

            calculated_parity = (compound_call_on_daughter.NPV() +
                                 discounted_mother_strike -
                                 compound_put_on_daughter.NPV() -
                                 vanilla_daughter_option.NPV())

            expected_parity = 0.0
            tolerance = 1.0e-8
            error = abs(calculated_parity - expected_parity)

            self.assertTrue(
                error <= tolerance,
                format_report_failure(
                    "put-call parity",
                    payoff_mother_call.optionType(), # Type of mother option used for CallOnX
                    payoff_daughter.optionType(),
                    strike_mother, strike_daughter, s, q, r,
                    self.todays_date, maturity_mother_date, maturity_daughter_date,
                    v, expected_parity, calculated_parity, error, tolerance
                )
            )

    def test_values(self):
        """Testing compound-option values and greeks."""
        print("Testing compound-option values and greeks...")

        # type Mother, typeDaughter, strike Mother, strike Daughter,  spot,    q,    r,    t Mother, t Daughter,  vol,   value,    tol, delta, gamma, vega, theta
        values_data = [
            { "tM": ql.Option.Put, "tD": ql.Option.Call,  "sM": 50.0, "sD": 520.0, "s": 500.0, "q": 0.03, "r": 0.08, "ttM": 0.25, "ttD": 0.5,  "v": 0.35, "npv": 21.1965, "tol": 1.0e-3, "delta": -0.1966, "gamma": 0.0007, "vega": -32.1241, "theta": -3.3837},
            { "tM": ql.Option.Call,"tD": ql.Option.Call,  "sM": 50.0, "sD": 520.0, "s": 500.0, "q": 0.03, "r": 0.08, "ttM": 0.25, "ttD": 0.5,  "v": 0.35, "npv": 17.5945, "tol": 1.0e-3, "delta":  0.3219, "gamma": 0.0038, "vega": 106.5185, "theta": -65.1614},
            { "tM": ql.Option.Call,"tD": ql.Option.Put,   "sM": 50.0, "sD": 520.0, "s": 500.0, "q": 0.03, "r": 0.08, "ttM": 0.25, "ttD": 0.5,  "v": 0.35, "npv": 18.7128, "tol": 1.0e-3, "delta": -0.2906, "gamma": 0.0036, "vega": 103.3856, "theta": -46.6982},
            { "tM": ql.Option.Put, "tD": ql.Option.Put,   "sM": 50.0, "sD": 520.0, "s": 500.0, "q": 0.03, "r": 0.08, "ttM": 0.25, "ttD": 0.5,  "v": 0.35, "npv": 15.2601, "tol": 1.0e-3, "delta":  0.1760, "gamma": 0.0005, "vega": -35.2570, "theta": -10.1126},
            { "tM": ql.Option.Call,"tD": ql.Option.Call,  "sM": 0.05, "sD": 1.14,  "s": 1.20,  "q": 0.0,  "r": 0.01, "ttM": 0.5,  "ttD": 2.0,  "v": 0.11, "npv": 0.0729,  "tol": 1.0e-3, "delta":  0.6614, "gamma": 2.5762, "vega":  0.5812, "theta": -0.0297},
            { "tM": ql.Option.Call,"tD": ql.Option.Put,   "sM": 0.05, "sD": 1.14,  "s": 1.20,  "q": 0.0,  "r": 0.01, "ttM": 0.5,  "ttD": 2.0,  "v": 0.11, "npv": 0.0074,  "tol": 1.0e-3, "delta": -0.1334, "gamma": 1.9681, "vega":  0.2933, "theta": -0.0155},
            { "tM": ql.Option.Put, "tD": ql.Option.Call,  "sM": 0.05, "sD": 1.14,  "s": 1.20,  "q": 0.0,  "r": 0.01, "ttM": 0.5,  "ttD": 2.0,  "v": 0.11, "npv": 0.0021,  "tol": 1.0e-3, "delta": -0.0426, "gamma": 0.7252, "vega": -0.0052, "theta": -0.0058},
            { "tM": ql.Option.Put, "tD": ql.Option.Put,   "sM": 0.05, "sD": 1.14,  "s": 1.20,  "q": 0.0,  "r": 0.01, "ttM": 0.5,  "ttD": 2.0,  "v": 0.11, "npv": 0.0192,  "tol": 1.0e-3, "delta":  0.1626, "gamma": 0.1171, "vega": -0.2931, "theta": -0.0028},
            { "tM": ql.Option.Call,"tD": ql.Option.Call,  "sM": 10.0, "sD": 122.0, "s": 120.0, "q": 0.06, "r": 0.02, "ttM": 0.1,  "ttD": 0.7,  "v": 0.22, "npv": 0.4419,  "tol": 1.0e-3, "delta":  0.1049, "gamma": 0.0195, "vega":  11.3368, "theta": -6.2871},
            { "tM": ql.Option.Call,"tD": ql.Option.Put,   "sM": 10.0, "sD": 122.0, "s": 120.0, "q": 0.06, "r": 0.02, "ttM": 0.1,  "ttD": 0.7,  "v": 0.22, "npv": 2.6112,  "tol": 1.0e-3, "delta": -0.3618, "gamma": 0.0337, "vega":  28.4843, "theta": -13.4124},
            { "tM": ql.Option.Put, "tD": ql.Option.Call,  "sM": 10.0, "sD": 122.0, "s": 120.0, "q": 0.06, "r": 0.02, "ttM": 0.1,  "ttD": 0.7,  "v": 0.22, "npv": 4.1616,  "tol": 1.0e-3, "delta": -0.3174, "gamma": 0.0024, "vega": -26.6403, "theta": -2.2720},
            { "tM": ql.Option.Put, "tD": ql.Option.Put,   "sM": 10.0, "sD": 122.0, "s": 120.0, "q": 0.06, "r": 0.02, "ttM": 0.1,  "ttD": 0.7,  "v": 0.22, "npv": 1.0914,  "tol": 1.0e-3, "delta":  0.1748, "gamma": 0.0165, "vega":  -9.4928, "theta": -4.8995},
            { "tM": ql.Option.Call,"tD": ql.Option.Call,  "sM": 0.4,  "sD": 8.2,   "s": 8.0,   "q": 0.05, "r": 0.00, "ttM": 2.0,  "ttD": 3.0,  "v": 0.08, "npv": 0.0099,  "tol": 1.0e-3, "delta":  0.0285, "gamma": 0.0688, "vega":   0.7764, "theta": -0.0027},
            { "tM": ql.Option.Call,"tD": ql.Option.Put,   "sM": 0.4,  "sD": 8.2,   "s": 8.0,   "q": 0.05, "r": 0.00, "ttM": 2.0,  "ttD": 3.0,  "v": 0.08, "npv": 0.9826,  "tol": 1.0e-3, "delta": -0.7224, "gamma": 0.2158, "vega":   2.7279, "theta": -0.3332},
            { "tM": ql.Option.Put, "tD": ql.Option.Call,  "sM": 0.4,  "sD": 8.2,   "s": 8.0,   "q": 0.05, "r": 0.00, "ttM": 2.0,  "ttD": 3.0,  "v": 0.08, "npv": 0.3585,  "tol": 1.0e-3, "delta": -0.0720, "gamma":-0.0835, "vega":  -1.5633, "theta": -0.0117},
            { "tM": ql.Option.Put, "tD": ql.Option.Put,   "sM": 0.4,  "sD": 8.2,   "s": 8.0,   "q": 0.05, "r": 0.00, "ttM": 2.0,  "ttD": 3.0,  "v": 0.08, "npv": 0.0168,  "tol": 1.0e-3, "delta":  0.0378, "gamma": 0.0635, "vega":   0.3882, "theta":  0.0021},
            { "tM": ql.Option.Call,"tD": ql.Option.Call,  "sM": 0.02, "sD": 1.6,   "s": 1.6,   "q": 0.013,"r": 0.022,"ttM": 0.45, "ttD": 0.5,  "v": 0.17, "npv": 0.0680,  "tol": 1.0e-3, "delta":  0.4937, "gamma": 2.1271, "vega":   0.4418, "theta": -0.0843},
            { "tM": ql.Option.Call,"tD": ql.Option.Put,   "sM": 0.02, "sD": 1.6,   "s": 1.6,   "q": 0.013,"r": 0.022,"ttM": 0.45, "ttD": 0.5,  "v": 0.17, "npv": 0.0605,  "tol": 1.0e-3, "delta": -0.4169, "gamma": 2.0836, "vega":   0.4330, "theta": -0.0697},
            { "tM": ql.Option.Put, "tD": ql.Option.Call,  "sM": 0.02, "sD": 1.6,   "s": 1.6,   "q": 0.013,"r": 0.022,"ttM": 0.45, "ttD": 0.5,  "v": 0.17, "npv": 0.0081,  "tol": 1.0e-3, "delta": -0.0417, "gamma": 0.0761, "vega":  -0.0045, "theta": -0.0020},
            { "tM": ql.Option.Put, "tD": ql.Option.Put,   "sM": 0.02, "sD": 1.6,   "s": 1.6,   "q": 0.013,"r": 0.022,"ttM": 0.45, "ttD": 0.5,  "v": 0.17, "npv": 0.0078,  "tol": 1.0e-3, "delta":  0.0413, "gamma": 0.0326, "vega":  -0.0133, "theta": -0.0016}
        ]

        greeks_to_test = ["npv", "delta", "gamma", "vega", "theta"]

        for val_dict in values_data:
            self._update_market_data(val_dict["s"], val_dict["q"], val_dict["r"], val_dict["v"])

            payoff_mother = ql.PlainVanillaPayoff(val_dict["tM"], val_dict["sM"])
            payoff_daughter = ql.PlainVanillaPayoff(val_dict["tD"], val_dict["sD"])

            # timeToDays(Time t) { return Natural(t*365.0); }
            maturity_mother_date = self.todays_date + int(val_dict["ttM"] * 365)
            maturity_daughter_date = self.todays_date + int(val_dict["ttD"] * 365)

            exercise_mother = ql.EuropeanExercise(maturity_mother_date)
            exercise_daughter = ql.EuropeanExercise(maturity_daughter_date)

            compound_option = ql.CompoundOption(
                payoff_mother, exercise_mother, payoff_daughter, exercise_daughter
            )
            compound_option.setPricingEngine(self.compound_engine)

            for greek_name in greeks_to_test:
                if greek_name == "npv":
                    calculated = compound_option.NPV()
                elif greek_name == "delta":
                    calculated = compound_option.delta()
                elif greek_name == "gamma":
                    calculated = compound_option.gamma()
                elif greek_name == "vega":
                    calculated = compound_option.vega()
                elif greek_name == "theta":
                    calculated = compound_option.theta()

                expected = val_dict[greek_name]
                tolerance = val_dict["tol"]
                error = abs(calculated - expected)

                self.assertTrue(
                    error <= tolerance,
                    format_report_failure(
                        greek_name, payoff_mother.optionType(), payoff_daughter.optionType(),
                        val_dict["sM"], val_dict["sD"], val_dict["s"],
                        val_dict["q"], val_dict["r"],
                        self.todays_date, maturity_mother_date, maturity_daughter_date,
                        val_dict["v"], expected, calculated, error, tolerance
                    )
                )


if __name__ == '__main__':
    print("Testing QuantLib " + ql.__version__)
    # Set evaluation date, if not using the default from Settings
    # For this test, we rely on TopLevelFixture's behavior which usually sets it.
    # eval_date = ql.Date(15, ql.May, 1998) # Example, if needed
    # ql.Settings.instance().evaluationDate = eval_date
    unittest.main(argv=['first-arg-is-ignored'], exit=False)